## notes for what the function should take
bootstrap_number, default is 100


network metric, either between centrality, connectivity


top and bottom which percent (default 10%)

current working directory


In [1]:
#deleting all to start a new bootstrap 
!rm -r conn*
!rm -r run*
!rm -r upper*
!rm -r lower*
!rm -r 0*
!rm -r bet*
!rm -r dfe_outfile_*
!rm -r output.txt
!rm -r prop_muts_in_s_ranges.out
!rm -r low_conn*
!rm -r alpha_omega_a_between_centrality.png
!rm -r random*

rm: cannot remove `conn*': No such file or directory
rm: cannot remove `upper*': No such file or directory
rm: cannot remove `lower*': No such file or directory
rm: cannot remove `0*': No such file or directory
rm: cannot remove `bet*': No such file or directory
rm: cannot remove `dfe_outfile_*': No such file or directory
rm: cannot remove `output.txt': No such file or directory
rm: cannot remove `prop_muts_in_s_ranges.out': No such file or directory
rm: cannot remove `low_conn*': No such file or directory
rm: cannot remove `alpha_omega_a_between_centrality.png': No such file or directory
rm: cannot remove `random*': No such file or directory


In [6]:
import os
for i in range(1,101):
    os.mkdir("./runs/run" + str(i))
    

In [4]:
import os 
os.chdir("/scratch/research/projects/chlamydomonas/network_evolution/analysis/August2020_Sara/bootstrap")
os.getcwd()

'/scratch/research/projects/chlamydomonas/network_evolution/analysis/August2020_Sara/bootstrap'

In [3]:
#bootstrapping 
from numpy import random as rd

# ary = ['ab', 'fgg', 'f', 'vb']
# keep_lst = (list(rd.choice(ary, size=8, replace=True)))
# cleaned = df[df[''].isin(remove_list)]
os.chdir("/scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/")

!cd /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/
!pwd

/scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap


In [7]:
import re
def get_dfe_outputs(filename, t):
    with open(filename) as f:
        content = f.readlines()[0]
        print(content)
        match = re.search('lambda (\d+.\d+) selected_divergence (\d+.\d+) alpha (\d+.\d+) omega_A (\d+.\d+)', content)
        with open(t+'_lambda.txt', 'a') as the_file:
            the_file.write("{}\n".format(match.group(1)))
        with open(t+'_selected_divergence.txt', 'a') as the_file:
            the_file.write("{}\n".format(match.group(2)))
        with open(t+'_alpha.txt', 'a') as the_file:
            the_file.write("{}\n".format(match.group(3)))
        with open(t+'_omega_A.txt', 'a') as the_file:
            the_file.write("{}\n".format(match.group(4)))

In [8]:
import pandas as pd
df = pd.read_table("new_485_genes_all_network_metrics.csv", sep=",")
print(df.columns)

Index(['Unnamed: 0', 'Unnamed: 0.1', 'SUID', 'AverageShortestPathLength',
       'BetweennessCentrality', 'ClosenessCentrality', 'ClusteringCoefficient',
       'Degree', 'Eccentricity', 'IsSingleNode', 'kineticLaw-',
       'kineticLaw--units', 'name', 'NeighborhoodConnectivity',
       'NumberOfDirectedEdges', 'NumberOfUndirectedEdges',
       'PartnerOfMultiEdgedNodePairs', 'Radiality', 'sbml charge',
       'sbml compartment', 'sbml id', 'sbml initial amount',
       'sbml initial concentration', 'sbml type', 'selected', 'SelfLoops',
       'shared name', 'Stress', 'TopologicalCoefficient', 'genename',
       'new_names', 'new_association', 'reinhardtii_gene', 'number',
       'Gene_name', 'differences_0', 'total_0', 'JC_distance_0',
       'differences_2', 'total_2', 'JC_distance_2', 'differences_4', 'total_4',
       'JC_distance_4', 'SFS0', 'SFS0_theta_pi', 'SFS4', 'SFS4_theta_pi',
       'connectivity', 'max_connectivity', 'sum_connectivity', 'new_name'],
      dtype='object')


In [42]:
%matplotlib inline
import pandas as pd
from itertools import product
import pandas as pd
import numpy as np
from collections import Counter
import collections
from numpy import random as rd
from ness_vcf import SFS

def sum_SFSs(list_of_SFSs):
    # for a bunch of SFSs that are the same length sum them alllll up into one big one\
#     sfs_size = len(list_of_SFSs[0])
#     for i in range(sfs_size):
#         sum([sfs[i] for sfs in list_of_SFSs ]
    total_SFS = [sum(i) for i in zip(*list_of_SFSs)]
    return total_SFS

def column2SFSs(column_of_SFSs):
    list_of_SFSs = []
    for s in list(column_of_SFSs):
        exec("list_of_SFSs.append(" + s + ")")
    return list_of_SFSs

def make_dfe_inputs(lower_path, upper_path):
    os.chdir("/scratch/research/projects/chlamydomonas/network_evolution/analysis/August2020_Sara/bootstrap/")
    df = pd.read_table('new_485_genes_all_network_metrics.csv', sep=",")
    
    x = "max_connectivity"
    cutoff_percent  = 0.10
    number_genes = df.shape[0]
    cutoff_number = round(number_genes*cutoff_percent)
    cutoff_high  = sorted(list(df[x]))[number_genes-cutoff_number]
    cutoff_low = sorted(list(df[x]))[cutoff_number]

    df_low = df[df[x] <= cutoff_low]
    df_high = df[df[x] >= cutoff_high]
    
    print(df_low)

    df_low_names = df_low['Gene_name'].tolist()
    random_low_lst = (list(rd.choice(df_low_names, size=100, replace=True)))  # cleaned = df[df['genename'].isin(keep_lst)] #ignores duplicates so this is wrong 
    low_newDF = pd.DataFrame() #creates a new dataframe that's empty
    for key,value in (dict(Counter(random_low_lst)).items()):
        for i in range(value):
                low_newDF = low_newDF.append(df_low.loc[df_low['Gene_name'] == key]) # ignoring index is optional

                
    df_high_names = df_high['Gene_name'].tolist()
    random_high_lst = (list(rd.choice(df_high_names, size=100, replace=True)))
    high_newDF = pd.DataFrame() #creates a new dataframe that's empty
    for key,value in (dict(Counter(random_high_lst)).items()):
        for i in range(value):
                high_newDF = high_newDF.append(df_high.loc[df_high['Gene_name'] == key]) # ignoring index is optional
        

    total_SFS0_low = SFS(sum_SFSs(column2SFSs(low_newDF.SFS0)))
    total_SFS4_low = SFS(sum_SFSs(column2SFSs(low_newDF.SFS4)))
    total_SFS0_high = SFS(sum_SFSs(column2SFSs(high_newDF.SFS0)))
    total_SFS4_high= SFS(sum_SFSs(column2SFSs(high_newDF.SFS4)))

    SFSs = [total_SFS0_low,total_SFS4_low,total_SFS0_high,total_SFS4_high]
    
    for s in SFSs:
        s.fold()

    sfs_lower_network_measure = "SFS.lower10th_connectivity.txt"
    sfs_
    SFS_lower_0 = " ".join(str(item) for item in SFSs[0].sfs)
    SFS_lower_1 = " ".join(str(item) for item in SFSs[1].sfs)
    !printf "1\n18\n" > $lower_path/SFS.lower10th_connectivity.txt
    !echo $SFS_lower_0 >> $lower_path/SFS.lower10th_connectivity.txt
    !echo $SFS_lower_1 >> $lower_path/SFS.lower10th_connectivity.txt
    
#     SFS_upper_0 = " ".join(str(item) for item in SFSs[2].sfs)
#     SFS_upper_1 = " ".join(str(item) for item in SFSs[3].sfs)
#     !printf "1\n18\n" > $upper_path/SFS.upper10th_connectivity.txt
#     !echo $SFS_upper_0 >> $upper_path/SFS.upper10th_connectivity.txt
#     !echo $SFS_upper_1 >> $upper_path/SFS.upper10th_connectivity.txt
    
    
#     low_sites_0 = sum(low_newDF.total_0)
#     low_diff_0 = sum(low_newDF.differences_0)
#     low_sites_4 = sum(low_newDF.total_4)
#     low_diff_4 = sum(low_newDF.differences_4)
#     !echo 1 $low_sites_0 $low_diff_0 > $lower_path/divergence.lower10th_connectivity.txt
#     !echo 0 $low_sites_4 $low_diff_4 >> $lower_path/divergence.lower10th_connectivity.txt

#     high_sites_0 = sum(high_newDF.total_0)
#     high_diff_0 = sum(high_newDF.differences_0)
#     high_sites_4 = sum(high_newDF.total_4)
#     high_diff_4 = sum(high_newDF.differences_4)
#     !echo 1 $high_sites_0 $high_diff_0 > $upper_path/divergence.upper10th_connectivity.txt
#     !echo 0 $high_sites_4 $high_diff_4 >> $upper_path/divergence.upper10th_connectivity.txt
    
    
#     print("perc Sites Diffs")
#     print("Low: ", sum(low_newDF.total_0), sum(low_newDF.differences_0))
#     print("Low: ", sum(low_newDF.total_4), sum(low_newDF.differences_4))
#     print("High: ", sum(high_newDF.total_0), sum(high_newDF.differences_0))
#     print("High: ", sum(high_newDF.total_4), sum(high_newDF.differences_4))

In [43]:
%%time
import os
rootdir = "/scratch/research/projects/chlamydomonas/network_evolution/analysis/August2020_Sara/bootstrap/runs"
print(rootdir)
i = 0
upper = "upper10th"
lower = "lower10th"
network_measure = "connectivity"
for subdir, dirs, files in os.walk(rootdir):
    for folder in dirs:
        if folder.startswith("r"):
            new_path = os.path.join(subdir, folder)
            print("this is the path " + str(new_path))
            os.chdir(new_path) #change directory to new_path 
            if not os.path.exists(upper) and not os.path.exists(lower):
                os.makedirs(upper)
                os.makedirs(lower)
                
            #make folders in lower folder for each bootstrap   
            os.chdir(new_path + "/" + lower) #change directory to new_path
            if not os.path.exists(lower + "_" + network_measure + "_neutral") and not os.path.exists(lower + "_" + network_measure + "_selected"):
                os.makedirs(lower + "_" + network_measure + "_neutral")
                os.makedirs(lower + "_" + network_measure + "_selected")
            
            os.chdir(new_path + "/" + upper) #change directory to new_path
            
            if not os.path.exists(upper + "_" + network_measure + "_neutral") and not os.path.exists(upper + "_" + network_measure + "_selected"):
                os.makedirs(upper + "_" + network_measure + "_neutral")
                os.makedirs(upper + "_" + network_measure + "_selected")



#             send the two paths (upper10th and lower10th to the function, to enter divergence and diversity )
            make_dfe_inputs(new_path + "/" + lower, new_path + "/" + upper)
            
#             ##lower 
#             !echo -e "data_path_1    /scratch/research/tmp_apps/dfe-alpha-release-2.16/data/ \nsfs_input_file    /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/lower10th/SFS.lower10th_connectivity.txt \nest_dfe_results_dir   /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/lower10th/lower10th_connectivity_neutral \nsite_class 0 \nfold       1 \nepochs     2 \nsearch_n2       1 \nt2_variable     1 \nt2              50" > /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/lower10th/run.lower10th.est_dfe-site_class0.config.txt            
            
#             ##upper
#             !echo -e "data_path_1    /scratch/research/tmp_apps/dfe-alpha-release-2.16/data/ \nsfs_input_file    /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/upper10th/SFS.upper10th_connectivity.txt \nest_dfe_results_dir   /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/upper10th/upper10th_connectivity_neutral \nsite_class 0 \nfold       1 \nepochs     2 \nsearch_n2       1 \nt2_variable     1 \nt2              50" > /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/upper10th/run.upper10th.est_dfe-site_class0.config.txt            
#             !bash dfe_first_step.sh /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/lower10th/run.lower10th.est_dfe-site_class0.config.txt
#             !bash dfe_first_step.sh /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/upper10th/run.upper10th.est_dfe-site_class0.config.txt
#             i+= 1
#             print("++++++++++++++++++++++++++++++++++++++*******" + str(i) + "*********************************************************************************" )
# #             #lower class1
#             !echo -e "data_path_1    /scratch/research/tmp_apps/dfe-alpha-release-2.16/data/ \nsfs_input_file    /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/lower10th/SFS.lower10th_connectivity.txt \nest_dfe_results_dir   /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/lower10th/lower10th_connectivity_selected \nest_dfe_demography_results_file    /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/lower10th/lower10th_connectivity_neutral/est_dfe.out \nsite_class    1 \nfold    1 \nepochs    2 \nmean_s_variable    1 \nmean_s    -0.1 \nbeta_variable    1 \nbeta    0.5" > /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/lower10th/run.lower10th.est_dfe-site_class1.config.txt            
#             !echo -e "data_path_1    /scratch/research/tmp_apps/dfe-alpha-release-2.16/data/ \nsfs_input_file    /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/upper10th/SFS.upper10th_connectivity.txt \nest_dfe_results_dir   /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/upper10th/upper10th_connectivity_selected \nest_dfe_demography_results_file    /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/upper10th/upper10th_connectivity_neutral/est_dfe.out \nsite_class    1 \nfold    1 \nepochs    2 \nmean_s_variable    1 \nmean_s    -0.1 \nbeta_variable    1 \nbeta    0.5" > /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/upper10th/run.upper10th.est_dfe-site_class1.config.txt            

#             !bash dfe_first_step.sh /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/lower10th/run.lower10th.est_dfe-site_class1.config.txt
#             !bash dfe_first_step.sh /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/upper10th/run.upper10th.est_dfe-site_class1.config.txt
            
#             !bash dfe_second_step.sh /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/upper10th/upper10th_connectivity_selected/est_dfe.out $folder/upperDFE.results.txt
#             !bash dfe_second_step.sh /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/lower10th/lower10th_connectivity_selected/est_dfe.out $folder/lowerDFE.results.txt
            
#             !echo -e "data_path_1    /scratch/research/tmp_apps/dfe-alpha-release-2.16/data/ \ndivergence_file         /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/lower10th/divergence.lower10th_connectivity.txt \nest_alpha_omega_results_file    /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/lower10th/est_alpha_omega.out \nest_dfe_results_file     /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/lower10th/lower10th_connectivity_selected/est_dfe.out \nneut_egf_file           /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/lower10th/lower10th_connectivity_neutral/neut_egf.out \nsel_egf_file            /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/lower10th/lower10th_connectivity_selected/sel_egf.out \ndo_jukes_cantor         1 \nremove_poly             1" > /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/lower10th/run.lower10th.est_alpha_omega.config.txt
#             !echo -e "data_path_1    /scratch/research/tmp_apps/dfe-alpha-release-2.16/data/ \ndivergence_file         /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/upper10th/divergence.upper10th_connectivity.txt \nest_alpha_omega_results_file    /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/upper10th/est_alpha_omega.out \nest_dfe_results_file     /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/upper10th/upper10th_connectivity_selected/est_dfe.out \nneut_egf_file           /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/upper10th/upper10th_connectivity_neutral/neut_egf.out \nsel_egf_file            /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/upper10th/upper10th_connectivity_selected/sel_egf.out \ndo_jukes_cantor         1 \nremove_poly             1" > /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/upper10th/run.upper10th.est_alpha_omega.config.txt

#             !bash testing.sh /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/upper10th/run.upper10th.est_alpha_omega.config.txt  
#             !bash testing.sh /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/lower10th/run.lower10th.est_alpha_omega.config.txt  
            
# #             /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/alpha_omega_upper_results.txt
# #             /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/$folder/alpha_omega_lower_results.txt
#             get_dfe_outputs('/scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/' + folder + '/upper10th/est_alpha_omega.out', "upper")
#             get_dfe_outputs('/scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/' + folder + '/lower10th/est_alpha_omega.out', "lower")



/scratch/research/projects/chlamydomonas/network_evolution/analysis/August2020_Sara/bootstrap/runs
this is the path /scratch/research/projects/chlamydomonas/network_evolution/analysis/August2020_Sara/bootstrap/runs/run84
     Unnamed: 0  Unnamed: 0.1  SUID  AverageShortestPathLength  \
8             8            28    90                   5.486257   
9             9          1576  1638                   4.512201   
11           11            37    99                   5.483689   
12           12            38   100                   5.484202   
13           13            39   101                   5.484716   
14           14          1407  1469                   5.486771   
15           15          1408  1470                   5.487285   
16           16          1409  1471                   5.487285   
17           17          1410  1472                   5.487285   
18           18          1411  1473                   5.487285   
19           19          1412  1474                  

/home/shawasar/.local/lib/python3.5/site-packages/ipykernel_launcher.py:27: FutureWarning: read_table is deprecated, use read_csv instead.


[<SFS.SFS object at 0x7f27e29f7128>, <SFS.SFS object at 0x7f27e29f7c50>, <SFS.SFS object at 0x7f27e29f7940>, <SFS.SFS object at 0x7f27e29f7630>]
this is the path /scratch/research/projects/chlamydomonas/network_evolution/analysis/August2020_Sara/bootstrap/runs/run56
     Unnamed: 0  Unnamed: 0.1  SUID  AverageShortestPathLength  \
8             8            28    90                   5.486257   
9             9          1576  1638                   4.512201   
11           11            37    99                   5.483689   
12           12            38   100                   5.484202   
13           13            39   101                   5.484716   
14           14          1407  1469                   5.486771   
15           15          1408  1470                   5.487285   
16           16          1409  1471                   5.487285   
17           17          1410  1472                   5.487285   
18           18          1411  1473                   5.487285   
19     

KeyboardInterrupt: 

In [1]:
import os
import re

def get_dfe_outputs(filename, t):
    with open(filename) as f:
        content = f.readlines()[0]
        return content
        match = re.search('0.000000 1.000000 (\d+.\d+) 1.000000 10.000000 (\d+.\d+) 10.000000 100.000000 (\d+.\d+) 100.000000 -99.000000 (\d+.\d+)', content)


rootdir = '/scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap'
i = 0

file_out_lower = open('connectivity_irc1080_outfile_lower.csv', 'w')
file_out_upper = open('connectivity_irc1080_outfile_upper.csv', 'w')

for subdir, dirs, files in os.walk(rootdir):
    for folder in dirs:
        if folder.startswith("./runs/r"):
            file_out_lower.write(get_dfe_outputs('/scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/' + folder + '/lowerDFE.results.txt', "lower"))
            file_out_upper.write(get_dfe_outputs('/scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/' + folder + '/upperDFE.results.txt', "upper"))

file_out_lower.close()
file_out_upper.close()